In [1]:
# pip install transformers
# pip install torch
# pip install scikit-learn tensorflow bs4

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
import requests
from bs4 import BeautifulSoup

In [3]:
# 그래프 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

In [4]:
# 네이버 API 인증 정보
client_id = 'YaKyDrFyIZDRfIiRAwBj'
client_secret = 'hknmr5WHGm'

In [5]:
# 업비트 데이터를 가져오는 함수
def fetch_upbit_data(market="KRW-BTC", count=200):
    url = "https://api.upbit.com/v1/candles/days"
    params = {"market": market, "count": count}
    response = requests.get(url, params=params)

    if response.status_code != 200:
        print(f"업비트 API에서 데이터를 가져오지 못했습니다. 상태 코드: {response.status_code}")
        return None

    data = response.json()
    df = pd.DataFrame(data)
    df = df[["candle_date_time_kst", "trade_price"]]
    df.columns = ["date", "price"]
    df["date"] = pd.to_datetime(df["date"]).dt.date
    df.sort_values(by="date", inplace=True)
    return df

In [6]:
# 뉴스 데이터를 가져오는 함수
def fetch_news_data(query, display=10):
    url = f"https://openapi.naver.com/v1/search/news.json?query={query}&display={display}"
    headers = {
        "X-Naver-Client-Id": client_id,
        "X-Naver-Client-Secret": client_secret
    }
    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        print(f"뉴스 API에서 데이터를 가져오지 못했습니다. 상태 코드: {response.status_code}")
        return None

    data = response.json()
    return pd.DataFrame(data['items'])

In [7]:
# 감정 분석을 위한 함수
def analyze_sentiment(texts):
    tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
    model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
    sentiments = []

    for text in texts:
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
        outputs = model(**inputs)
        sentiment = torch.argmax(outputs.logits, dim=1).item() + 1  # 감정 점수: 1~5
        sentiments.append(sentiment)

    return sentiments

In [8]:
# URL에서 뉴스 제목과 본문을 크롤링하는 함수
def crawl_additional_news(url):
    response = requests.get(url)
    if response.status_code != 200:
        print(f"뉴스 크롤링 실패: 상태 코드 {response.status_code}")
        return None, None

    soup = BeautifulSoup(response.text, "html.parser")

    # 뉴스 제목 크롤링
    title_tag = soup.find("h1")  # 실제 태그와 클래스명을 확인 후 수정
    if title_tag is None:
        title = "제목 없음"
    else:
        title = title_tag.text.strip()

    # 뉴스 본문 크롤링
    content_tag = soup.find("div", class_="content")  # 실제 태그와 클래스명을 확인 후 수정
    if content_tag is None:
        content = "본문 없음"
    else:
        content = content_tag.text.strip()

    return title, content

In [9]:
# BTC/KRW 시장 데이터 가져오기
df_price = fetch_upbit_data()
if df_price is None:
    raise ValueError("업비트 데이터를 가져오지 못했습니다.")

In [11]:
# from datetime import datetime, timedelta

# # 미래 예측 날짜 생성
# def generate_future_dates(start_date, days):
#     return [start_date + timedelta(days=i) for i in range(1, days+1)]

# # 금요일까지 예측할 날짜 생성
# latest_date = df_price['date'].max()
# future_dates = generate_future_dates(latest_date, 3)  # 3일 뒤까지 예측

# # 미래 데이터를 위한 빈 값 추가
# future_df = pd.DataFrame({
#     'date': future_dates,
#     'price': [np.nan] * len(future_dates),
#     'average_sentiment': [2.5] * len(future_dates)  # 감정 점수는 중립으로 설정
# })

# # 기존 데이터와 미래 데이터를 병합
# merged_df = pd.concat([merged_df, future_df], ignore_index=True)


In [12]:
# 뉴스 데이터 가져오기
news_df = fetch_news_data("비트코인")
if news_df is None:
    raise ValueError("뉴스 데이터를 가져오지 못했습니다.")

In [13]:
# 뉴스 데이터 날짜별 처리
news_df['pubDate'] = pd.to_datetime(news_df['pubDate'], errors='coerce')
news_df['date'] = news_df['pubDate'].dt.date
news_df = news_df.dropna(subset=['date'])

In [14]:
# 감정 분석 수행
news_df['sentiment'] = analyze_sentiment(news_df['title'])

In [15]:
# 크롤링할 뉴스 URL 리스트
news_urls = [
    "https://www.g-enews.com/ko-kr/news/article/news_all/202312031530096281e7e8286d56_1/article.html",
    "https://www.yna.co.kr/view/AKR20231007023200002?input=1195m",
    "https://www.hankyung.com/article/2023100952301",
    # 추가 URL을 여기에 입력 가능
]

In [16]:
#추가 뉴스 크롤링 및 데이터 추가
for url in news_urls:
    additional_title, additional_content = crawl_additional_news(url)
    if additional_title:
        additional_sentiment = analyze_sentiment([additional_title])[0]
        print(f"추가 뉴스 제목: {additional_title}")
        print(f"추가 뉴스 감정 점수: {additional_sentiment}")
        # 추가된 뉴스 데이터를 기존 데이터프레임에 병합
        news_df = pd.concat([news_df, pd.DataFrame({"date": [pd.to_datetime("2023-12-03").date()],
                                                    "title": [additional_title],
                                                    "sentiment": [additional_sentiment]})], ignore_index=True)


추가 뉴스 제목: 일론 머스크 관련 ‘밈 토큰’ 급등…가상화폐 생태계 흔들까
추가 뉴스 감정 점수: 3
추가 뉴스 제목: 상반기 국내 가상자산 시총 28조…비트코인 등 회복세에 46%↑
추가 뉴스 감정 점수: 1
추가 뉴스 제목: 금융
추가 뉴스 감정 점수: 4


In [17]:
# 날짜별 평균 감정 점수 계산
sentiment_by_date = news_df.groupby('date')['sentiment'].mean().reset_index()
sentiment_by_date.columns = ['date', 'average_sentiment']

In [18]:
# 감정 점수 병합
merged_df = pd.merge(df_price, sentiment_by_date, on='date', how='left')
merged_df['average_sentiment'].fillna(2.5, inplace=True)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_31368\731998398.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df['average_sentiment'].fillna(2.5, inplace=True)


In [19]:
from datetime import datetime, timedelta

# 미래 예측 날짜 생성
def generate_future_dates(start_date, days):
    return [start_date + timedelta(days=i) for i in range(1, days+1)]

# 금요일까지 예측할 날짜 생성
latest_date = df_price['date'].max()
future_dates = generate_future_dates(latest_date, 3)  # 3일 뒤까지 예측

# 미래 데이터를 위한 빈 값 추가
future_df = pd.DataFrame({
    'date': future_dates,
    'price': [np.nan] * len(future_dates),
    'average_sentiment': [2.5] * len(future_dates)  # 감정 점수는 중립으로 설정
})

# 기존 데이터와 미래 데이터를 병합
merged_df = pd.concat([merged_df, future_df], ignore_index=True)


In [20]:
# 데이터 스케일링
scaler = MinMaxScaler()
scaler.fit(df_price[['price']])
merged_df['scaled_price'] = scaler.transform(merged_df['price'].values.reshape(-1, 1))
merged_df['scaled_sentiment'] = scaler.transform(merged_df['average_sentiment'].values.reshape(-1, 1))

C:\Users\isaac\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
C:\Users\isaac\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [21]:
# 멀티모달 데이터 생성 함수 정의
def create_multimodal_sequences(prices, sentiments, seq_length):
    sequences = []
    labels = []
    for i in range(len(prices) - seq_length):
        seq = np.column_stack((prices[i:i + seq_length], sentiments[i:i + seq_length]))
        label = prices[i + seq_length]
        sequences.append(seq)
        labels.append(label)
    return np.array(sequences), np.array(labels)

In [22]:
# 멀티모달 데이터 준비
sequence_length = 10
X, y = create_multimodal_sequences(
    merged_df['scaled_price'].values,
    merged_df['scaled_sentiment'].values,
    sequence_length
)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [23]:
# GRU 모델 생성
model = Sequential([
    GRU(128, input_shape=(sequence_length, 2), return_sequences=True),
    Dropout(0.2),
    GRU(64),
    Dropout(0.2),
    Dense(1)
])
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

C:\Users\isaac\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
# 모델 학습
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=20,
    batch_size=16,
    callbacks=[EarlyStopping(patience=5, restore_best_weights=True)]
)

Epoch 1/20


In [ ]:
# 예측값과 실제값 복원
predictions = model.predict(X_test)
original_scale_predictions = scaler.inverse_transform(predictions)
original_scale_y_test = scaler.inverse_transform(y_test.reshape(-1, 1))

In [ ]:
# 결과 출력
print("샘플 예측값과 실제값:")
for pred, true in zip(original_scale_predictions[:5], original_scale_y_test[:5]):
    print(f"예측값: {pred[0]:,.2f}, 실제값: {true[0]:,.2f}")

In [ ]:
# 뉴스 감정 분석 결과 시각화
sentiment_counts = news_df['sentiment'].value_counts().sort_index()
categories = ['부정', '중립', '긍정']
counts = [
    sentiment_counts.get(1, 0) + sentiment_counts.get(2, 0),
    sentiment_counts.get(3, 0),
    sentiment_counts.get(4, 0) + sentiment_counts.get(5, 0)
]
plt.figure(figsize=(8, 6))
plt.bar(categories, counts, color=['red', 'gray', 'green'])
plt.title('뉴스 감정 분석 결과 분포')
plt.xlabel('감정')
plt.ylabel('빈도')
plt.show()

In [ ]:
# 예측값과 실제값 비교 시각화
plt.figure(figsize=(10, 6))
plt.plot(merged_df['date'][-len(original_scale_y_test):], original_scale_y_test, label='실제 가격', color='blue', alpha=0.6)
plt.plot(merged_df['date'][-len(original_scale_predictions):], original_scale_predictions, label='예측 가격', color='red', alpha=0.6)

# Y축 포맷 설정
plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: f'{int(x):,}'))
plt.title('실제 비트코인 가격과 예측 가격 비교')
plt.xlabel('날짜')
plt.ylabel('가격 (KRW)')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
results_df = pd.DataFrame({
    "Actual Price": original_scale_y_test.flatten(),
    "Predicted Price": original_scale_predictions.flatten()
})


In [ ]:
# 2. 날짜별 비트코인 가격과 감정 점수 정리
merged_df_sorted = merged_df.sort_values("date")
price_and_sentiment_df = merged_df_sorted[["date", "price", "average_sentiment"]]

In [ ]:

# 1. 예측 값과 실제 값 비교 그래프
plt.figure(figsize=(12, 6))
plt.plot(results_df.index, results_df["Actual Price"], label="Actual Price", linestyle="-", marker="o")
plt.plot(results_df.index, results_df["Predicted Price"], label="Predicted Price", linestyle="--", marker="x")
plt.title("Actual vs Predicted Bitcoin Prices")
plt.xlabel("Sample Index")
plt.ylabel("Price (KRW)")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# 비트코인이 오를지 안 오를지 판단하는 함수 추가
def predict_bitcoin_trend(predictions, actual_values):
    """
    예측값과 실제값을 기반으로 비트코인의 상승 또는 하락 여부를 판단
    """
    if len(predictions) == 0 or len(actual_values) == 0:
        return "예측 데이터를 확인할 수 없습니다."

    latest_prediction = predictions[-1][0]  # 가장 최신 예측값
    latest_actual = actual_values[-1][0]    # 가장 최신 실제값

    if latest_prediction > latest_actual:
        return f"비트코인의 가격이 오를 것으로 예측됩니다. (예측값: {latest_prediction:,.2f} KRW)"
    elif latest_prediction < latest_actual:
        return f"비트코인의 가격이 내릴 것으로 예측됩니다. (예측값: {latest_prediction:,.2f} KRW)"
    else:
        return "비트코인의 가격이 변동이 없을 것으로 예측됩니다."

# 예측값과 실제값 복원
predictions = model.predict(X_test)
original_scale_predictions = scaler.inverse_transform(predictions)
original_scale_y_test = scaler.inverse_transform(y_test.reshape(-1, 1))

# 결과 출력
print("샘플 예측값과 실제값:")
for pred, true in zip(original_scale_predictions[:5], original_scale_y_test[:5]):
    print(f"예측값: {pred[0]:,.2f}, 실제값: {true[0]:,.2f}")

# 비트코인 상승/하락 여부 출력
trend_prediction = predict_bitcoin_trend(original_scale_predictions, original_scale_y_test)
print(trend_prediction)

금요일을 위해 추가된 부분

In [ ]:
# 금요일까지 예측을 위한 미래 날짜 생성 및 시퀀스 추가 (추가된 부분)
from datetime import timedelta

# 마지막 날짜로부터 금요일까지 날짜 생성
latest_date = merged_df['date'].max()
future_dates = []
while latest_date.weekday() < 4:  # 금요일의 weekday()는 4
    latest_date += timedelta(days=1)
    future_dates.append(latest_date)

In [ ]:
# 미래 날짜를 위한 빈 데이터프레임 생성
future_df = pd.DataFrame({
    'date': future_dates,
    'price': [np.nan] * len(future_dates),
    'average_sentiment': [2.5] * len(future_dates)  # 감정 점수를 중립값으로 설정
})

In [ ]:
# 기존 데이터와 결합
future_df['scaled_price'] = scaler.transform(np.array([merged_df['price'].iloc[-1]] * len(future_df)).reshape(-1, 1))
future_df['scaled_sentiment'] = 0.5  # 중립 감정 점수를 스케일링

In [ ]:
# 예측용 시퀀스 생성
future_X = []
last_sequence = merged_df[['scaled_price', 'scaled_sentiment']].values[-sequence_length:]

for i in range(len(future_dates)):
    seq = np.vstack([last_sequence[1:], [future_df['scaled_price'].iloc[i], future_df['scaled_sentiment'].iloc[i]]])
    future_X.append(seq)
    last_sequence = seq

future_X = np.array(future_X)

In [ ]:
# 모델을 사용해 미래 예측 수행
future_predictions = model.predict(future_X)
future_predictions_original_scale = scaler.inverse_transform(future_predictions)

In [ ]:
# 결과 병합
future_df['Predicted Price'] = future_predictions_original_scale.flatten()

In [ ]:
# 예측 결과 출력
print("금요일까지의 예측 결과:")
print(future_df[['date', 'Predicted Price']])

In [ ]:
# 결과 출력 (샘플 예측값과 실제값)
print("\n샘플 예측값과 실제값:")
for pred, true in zip(future_predictions_original_scale[:5], [merged_df['price'].iloc[-1]] * 5):
    print(f"예측값: {pred[0]:,.2f}, 실제값: {true:,.2f}")

In [ ]:
# 비트코인 상승/하락 여부 판단 함수
def predict_bitcoin_trend(predictions, last_actual):
    """
    금요일까지 예측값과 가장 마지막 실제값을 기반으로 비트코인 상승/하락 여부 판단
    """
    latest_prediction = predictions[-1][0]  # 가장 최신 예측값
    if latest_prediction > last_actual:
        return f"비트코인의 가격이 오를 것으로 예측됩니다. (예측값: {latest_prediction:,.2f} KRW)"
    elif latest_prediction < last_actual:
        return f"비트코인의 가격이 내릴 것으로 예측됩니다. (예측값: {latest_prediction:,.2f} KRW)"
    else:
        return "비트코인의 가격이 변동이 없을 것으로 예측됩니다."

In [ ]:
# 비트코인 상승/하락 여부 판단 및 출력
trend_prediction = predict_bitcoin_trend(future_predictions_original_scale, merged_df['price'].iloc[-1])
print("\n" + trend_prediction)

In [ ]:
# 시각화 (추가된 부분)
plt.figure(figsize=(10, 6))
plt.plot(merged_df['date'], merged_df['price'], label='Actual Price', linestyle='-', marker='o')
plt.plot(future_df['date'], future_df['Predicted Price'], label='Predicted Price', linestyle='--', color='red', marker='x')
plt.title('비트코인 실제 가격과 금요일까지의 예측 가격')
plt.xlabel('날짜')
plt.ylabel('가격 (KRW)')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()